<img style="float: left;" src="figures/DFN.png" width="10%">   

# <font color='Red'> $\;$ A geothermal doublet in a fractured reservoir</font>

## Import all important packages

It includes two local files:
 * [Model](https://gitlab.com/open-darts/darts-models/-/blob/development/teaching/EAGE/model.py) with main model description
 * [Model_input](https://gitlab.com/open-darts/darts-models/-/blob/development/teaching/EAGE/model_input.py) with input parameters


In [ ]:
import os
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt

from darts.engines import redirect_darts_output
from darts.tools.fracture_network.preprocessing_code import frac_preprocessing

from model import Model
from model_input import input_data_default

redirect_darts_output('dfm_model.log')

## Set parameters

In [ ]:
def input_data_case():
    input_data = input_data_default()
    
    input_data['case_name'] = 'case_1'

    # geometry
    input_data['frac_file'] = 'frac.txt' # file with fractures tips coordinates (X1, Y1, X2, Y2)

    # cell sizes
    input_data['char_len'] = 20  # near fractures (characteristic length for cleaning and mesh generation) [m]
    input_data['char_len_boundary'] = 150  # grid size near grid boundaries [m]

    input_data['z_top'] = 2000  # top depth of the reservoir [m]
    input_data['height'] = 10  # reservoir thickness [m]

    input_data['frac_aper'] = 1e-3  # (initial) fracture aperture [m]
    
    input_data['perm'] = 1 # [mD]
    
    input_data['hcap'] = 2200. # [kJ/m3/K]
    input_data['conduction'] = 181.44  # [kJ/m/day/K]
    
    # uniform initial pressure and temperature
    input_data['initial_uniform'] = True
    input_data['uniform_pressure'] = 250.  # [bars]
    input_data['uniform_temperature'] = 380.  # [K]

    # well locations; # X, Y, Z (only one perforation)
    input_data['inj_well_coords'] = [[200, 200, 2000]]  
    input_data['prod_well_coords'] = [[800, 800, 2000]]

    # well controls. the difference between the initial reservoir p,T and wells
    input_data['delta_temp'] = 40    # [K]      inj_temperature = initial_temp - delta_temp
    input_data['delta_p_inj']  = 20  # [bars]. inj_bhp = initial_pressure + delta_p_inj
    input_data['delta_p_prod'] = 20  # [bars]. inj_prod = initial_pressure - delta_p_prod

    return input_data

In [ ]:
input_data = input_data_case()

## Plot Discrete Fractured Network (DFN)

In [ ]:
def plot_dfn(input_data):
    frac_data_raw = np.genfromtxt(input_data['frac_file'])

    plt.gca().set_aspect('equal')
    for i in range(frac_data_raw.shape[0]):
        plt.plot(np.append(frac_data_raw[i, 0], frac_data_raw[i, 2]),
                 np.append(frac_data_raw[i, 1], frac_data_raw[i, 3]))
    
    wells_inj = input_data['inj_well_coords']
    plt.plot(wells_inj[0][0], wells_inj[0][1], 'o', color='b', label='inj well')
    wells_prod = input_data['prod_well_coords']
    plt.plot(wells_prod[0][0], wells_prod[0][1], 'o', color='r', label='prod well')
    
    plt.xlabel('X, m.')
    plt.ylabel('Y, m.')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plot_dfn(input_data)

## Mesh generation

For details on DFN mesh generation and parameters see https://doi.org/10.1029/2021WR030743.

In [ ]:
# Read fracture tips from input_data['frac_file'] and generate a .geo text file (input for gmsh), then
# call gmesh to create a mesh and output it to .msh text file, which will be used as an input to DARTS
# These files are stored tin the 'meshes' folder, one mesh is original (raw) 
# and the second is optimized for calculation (cleaned)

def generate_mesh(input_data):
    output_dir = 'meshes'
    frac_data_raw = np.genfromtxt(input_data['frac_file'])

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Input parameters for cleaning procedure
    angle_tol_straighten = 2  # tolerance for straightening fracture segments [degrees]
    merge_threshold = 0.5  # tolerance for merging nodes in algebraic constraint, values on interval [0.5, 0.86] [-]
    angle_tol_remove_segm = np.arctan(0.15) * 180 / np.pi   # tolerance for removing accute intersections [degrees]
    decimals = 7  # in order to remove duplicates we need to have fixed number of decimals
    num_partition_x = 4  # number of partitions for parallel implementation of intersection finding algorithm
    num_partition_y = 4  # " ... "

    frac_preprocessing(frac_data_raw, char_len=input_data['char_len'], output_dir=output_dir, 
                       filename_base=input_data['case_name'], merge_threshold=merge_threshold, z_top=input_data['z_top'],
                       height_res=input_data['height'], angle_tol_small_intersect=angle_tol_remove_segm, 
                       apertures_raw=None, box_data=input_data['box_data'], margin=input_data['margin'],
                       mesh_clean=input_data['mesh_clean'], mesh_raw=False, 
                       angle_tol_straighten=angle_tol_straighten, straighten_after_cln=True, decimals=decimals,
                       tolerance_zero=1e-10, tolerance_intersect=1e-10, calc_intersections_before=False, 
                       calc_intersections_after=False, num_partition_x=num_partition_x, num_partition_y=num_partition_y, 
                       partition_fractures_in_segms=True, matrix_perm=1, correct_aperture=False,
                       small_angle_iter=2, char_len_mult=1, char_len_boundary=input_data['char_len_boundary'], 
                       main_algo_iters=1, wells=None, char_len_well=input_data['char_len_well'], input_data=input_data)

In [ ]:
# need gmsh installed and callable from command line in order to mesh or gmsh python package installed
generate_mesh(input_data)

## Run simulation

In [ ]:
m = Model(input_data)
m.init(output_folder='output_3')
m.set_sim_params(first_ts=1e-2, mult_ts=2, max_ts=60, tol_newton=1e-3, tol_linear=1e-4, it_newton=10, it_linear=50)

# output initial solution to vtk file
output_dir = 'vtk_output_dfn'

# run simulation for 365 days
m.run(365)

# output current timestep to vtk file
m.output_to_vtk(ith_step=1, output_directory=output_dir)

## Plot temperature and rates

In [ ]:
def plot_wells(pkl_fname, axx, plot_cols):
    time_data = pickle.load(open(pkl_fname, 'rb'))

    td = time_data
    for k in td.keys():
        if 'temperature' in k:
            td[k] -= 273.15
            td = td.rename(columns={k: k.replace('(K)', '(degrees)')})
        else: # for rates
            td[k] = np.abs(td[k])

    # plot the defined columns for all wells
    for i, col in enumerate(plot_cols):
        y = td.filter(like=col).columns.to_list()

        td.plot(x='Time (years)', y=y, ax=axx[i])
        axx[i].set_ylabel('%s %s'%(col, td.filter(like=col).columns.tolist()[0].split(' ')[-1]))
        l = labels=[lab.split(':')[0].split('(')[0] for lab in axx[i].get_legend_handles_labels()[1]]
        axx[i].legend(l,frameon=False, ncol=2)
        axx[i].tick_params(axis=u'both', which=u'both',length=0)
        for location in ['top','bottom','left','right']:
            axx[i].spines[location].set_linewidth(0)
            axx[i].grid(alpha=0.3)
            
    plt.tight_layout()

In [ ]:
# postprocessing

# save well data to pkl
time_data = pd.DataFrame.from_dict(m.physics.engine.time_data)
time_data['Time (years)'] = time_data['time']/365.
pkl_fname = 'time_data.pkl'
pickle.dump(time_data, open(pkl_fname, 'wb'))

# plot two variables from pkl file
plot_cols = ['temperature', 'water rate']
fig, ax = plt.subplots(1, len(plot_cols), figsize=(12,5))
plot_wells(pkl_fname, fig.axes, plot_cols)
plt.show()

## Plot temperature map

In [ ]:
def plot_pyvista(input_data, output_dir):
    import pyvista as pv

    vtk_fname = os.path.join(output_dir, 'solution0.vtk')

    # get vts data
    mesh = pv.read(vtk_fname)

    # define plotter
    plotter = pv.Plotter()

    # set temperature as active scalar
    temp = mesh.set_active_scalars('T,degrees')
    # add threshold levels
    thresT = mesh.threshold([60, 110], invert=False)
    
    # set fracture as active scalar
    mesh.set_active_scalars('matrix_cell_bool')
    # plot only fractures (0 index)
    thresF = mesh.threshold([0, 0], invert=False)
    # add outline of mesh
    outline = mesh.outline()

    # add elements to plotter
    plotter.set_background('#52576c')
    plotter.add_mesh(outline, color='k')
    try: # in case there is an issue in PyVista "'NoneType' object is not callable"
        plotter.add_mesh(thresT, cmap='coolwarm', opacity=0.99,
                     scalar_bar_args={'title':'Temperature (\N{DEGREE SIGN}C)'})
        plotter.add_mesh(thresF, show_scalar_bar=False, cmap='coolwarm')
    except:
        plotter.add_mesh(thresT, opacity=0.99,
                     scalar_bar_args={'title':'Temperature (\N{DEGREE SIGN}C)'})
        plotter.add_mesh(thresF, show_scalar_bar=False)
    
    # add wells as lines
    wells_inj = input_data['inj_well_coords']
    wells_prod = input_data['prod_well_coords']

    i = 0

    injline = np.array([[wells_inj[i][0], wells_inj[i][1], 2000], [wells_inj[i][0], wells_inj[i][1], 2200]])
    prodline = np.array([[wells_prod[i][0], wells_prod[i][1], 2000], [wells_prod[i][0], wells_prod[i][1], 2200]])

    plotter.add_lines(injline, color='b', name='injector')
    plotter.add_lines(prodline, color='r', name='producer')
    plotter.add_axes(line_width=5, labels_off=False)
    plotter.camera_position = [-2,-5,5] 
    
    plotter.show()

In [ ]:
# plot temperature map from vtk file
plot_pyvista(input_data, output_dir)

## <font color='Blue'>Tasks in this workshop:</font>

Plot and copy production temperature and rates after each task items:

1. Run simulation for 10 years and compare well output and the temperature map
2. Change the reservoir thickness to 100 m and compare well output 
3. Change matrix permeability to granite (0.01 mD) and compare well output
4. Change matrix permeability to sandstone (100 mD) and compare well output

More details on modeling of geothermal energy production in fractured reservoirs can be found in https://doi.org/10.1016/j.advwatres.2021.103985 